<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/stats/consolidate_weather_data_run_sql_spark_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls drive/MyDrive/weather


 2023-08-06T07:28:40Z.waves.json   2024-08-11T10:44:14Z.waves.json   paphos2024-07-30.csv
 2023-08-07T03:02:07Z.waves.json   2024-08-12T04:48:07Z.waves.json   paphos2024-07-31.csv
 2023-08-08T03:45:13Z.waves.json   2024-08-13T02:33:41Z.waves.json   paphos2024-08-01.csv
 2023-08-09T02:27:07Z.waves.json   2024-08-14T04:41:42Z.waves.json   paphos2024-08-01.gsheet
 2023-08-10T07:23:49Z.waves.json   2024-08-15T04:49:59Z.waves.json   paphos2024-08-02.csv
 2023-08-11T03:05:31Z.waves.json   2024-08-16T13:17:25Z.waves.json   paphos2024-08-03.csv
 2023-08-12T10:44:14Z.waves.json   2024-08-17T03:52:20Z.waves.json   paphos2024-08-04.csv
 2023-08-13T04:48:07Z.waves.json   2024-08-18T13:28:30Z.waves.json   paphos2024-08-05.csv
 2023-08-14T02:33:41Z.waves.json   2024-08-19T03:06:16Z.waves.json   paphos2024-08-06.csv
 2023-08-15T04:41:42Z.waves.json   2024-08-20T02:15:52Z.waves.json   paphos2024-08-07.csv
 2023-08-16T04:49:59Z.waves.json   2024-08-21T07:08:36Z.waves.json   paphos2024-08-08.csv
 2023-0

In [5]:
import os
import pandas as pd

# Define the directory containing the CSV files
directory = 'drive/MyDrive/weather'

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with 'paphos2024' and ends with '.csv'
    if filename.startswith('paphos2024') and filename.endswith('.csv'):
        # Read the CSV file into a DataFrame
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        dataframes.append(df)

# Combine all DataFrames into one
df_combined = pd.concat(dataframes, ignore_index=True)

# Drop duplicate rows
df_combined = df_combined.drop_duplicates()

# Save the combined DataFrame to a new CSV file (optional)
df_combined.to_csv('combined_paphos2024.csv', index=False)

# Print a summary of the combined DataFrame
print(f"Combined DataFrame shape: {df_combined.shape}")

Combined DataFrame shape: (9222, 24)


In [7]:
from pyspark.sql import SparkSession



# Initialize SparkSession
spark = SparkSession.builder \
    .appName("weather") \
    .getOrCreate()

df = spark.read.csv(
    "combined_paphos2024.csv",
    header=True,        # Use the first row as column names
    inferSchema=True,   # Automatically infer data types
    sep=",",            # Specify delimiter (default is ',')
    encoding="UTF-8"    # Handle encoding
)



In [ ]:
df.columns

In [8]:
df.createOrReplaceTempView("weather")

In [9]:

sql = '''
SELECT SUM(precip) AS total_precip, MONTH(datetime) AS month
FROM weather
GROUP BY MONTH(datetime)

'''

result = spark.sql(sql)
result.show()

+------------------+-----+
|      total_precip|month|
+------------------+-----+
|259.33699999999976|   12|
|1.6430000000000005|    9|
|             0.008|    8|
|               0.0|    7|
|1.5000000000000002|   10|
|166.32799999999978|   11|
+------------------+-----+

